In [160]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import datetime as dt

In [161]:
songs=pd.read_csv('ml-03-data-processing-songs-dataset.csv')
songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Index                   1994 non-null   int64  
 1   Title                   1991 non-null   object 
 2   Artist                  1986 non-null   object 
 3   Top Genre               1986 non-null   object 
 4   Year                    1987 non-null   float64
 5   Month                   1994 non-null   object 
 6   Beats Per Minute (BPM)  1985 non-null   float64
 7   Energy                  1986 non-null   float64
 8   Danceability            970 non-null    float64
 9   Loudness (dB)           1987 non-null   float64
 10  Liveness                1986 non-null   float64
 11  Valence                 960 non-null    float64
 12  Length (Duration)       1985 non-null   object 
 13  Acousticness            1986 non-null   float64
 14  Speechiness             1990 non-null   

In [162]:
### 1. Data Hygiene ###

#Month - Convert str rows to numeric after converting 3 letter abbreviation to numerics (i.e. Jan->1)
songs['Month']=songs['Month'].apply(lambda x: dt.datetime.strptime(x,'%b').month if not x.isnumeric() else int(x))

#Length - Convert str rows to numeric after removing commas
songs['Length (Duration)']=songs['Length (Duration)'].apply(lambda x: float(x.replace(',','')) if isinstance(x,str) else x)

#Year - There is a "92" that is a recording error
songs['Year']=songs['Year'].apply(lambda x: x+1900 if x==92 else x)

songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Index                   1994 non-null   int64  
 1   Title                   1991 non-null   object 
 2   Artist                  1986 non-null   object 
 3   Top Genre               1986 non-null   object 
 4   Year                    1987 non-null   float64
 5   Month                   1994 non-null   int64  
 6   Beats Per Minute (BPM)  1985 non-null   float64
 7   Energy                  1986 non-null   float64
 8   Danceability            970 non-null    float64
 9   Loudness (dB)           1987 non-null   float64
 10  Liveness                1986 non-null   float64
 11  Valence                 960 non-null    float64
 12  Length (Duration)       1985 non-null   float64
 13  Acousticness            1986 non-null   float64
 14  Speechiness             1990 non-null   

In [163]:
### 2. Missing Values ###

#Remove rows where at least 50% of the values are missing
songs.dropna(axis=0,thresh=len(songs.columns)/2,inplace=True)

#Remove columns where at least 50% of the values are missing
songs.dropna(axis=1,thresh=len(songs)/2,inplace=True)

songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987 entries, 0 to 1993
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Index                   1987 non-null   int64  
 1   Title                   1987 non-null   object 
 2   Artist                  1984 non-null   object 
 3   Top Genre               1983 non-null   object 
 4   Year                    1984 non-null   float64
 5   Month                   1987 non-null   int64  
 6   Beats Per Minute (BPM)  1983 non-null   float64
 7   Energy                  1983 non-null   float64
 8   Loudness (dB)           1983 non-null   float64
 9   Liveness                1983 non-null   float64
 10  Length (Duration)       1982 non-null   float64
 11  Acousticness            1984 non-null   float64
 12  Speechiness             1986 non-null   float64
 13  Popularity              1985 non-null   float64
dtypes: float64(9), int64(2), object(3)
memor

In [164]:
### 3. Descriptive Statistics/Outliers ###
outliers={} #A dictionary to easily see which columns have outliers and what values the outliers are
for column in songs.columns:
    if column in ["Index","Title","Artist","Top Genre"]:
        continue
    else:
        minflag=songs[column].mean()-3*songs[column].std()
        maxflag=songs[column].mean()+3*songs[column].std()
        if songs[column].min()<minflag or songs[column].max()>maxflag:            
            outliers[column]=songs[(songs[column]>maxflag)|(songs[column]<minflag)][column].sort_values().values
            print(f"Nonoutlier range in {column}: ({round(minflag,2)}, {round(maxflag,2)})")
            print('Outlier List:')
            print(outliers[column])
            print('')
        else:
            continue

Nonoutlier range in Beats Per Minute (BPM): (34.31, 206.15)
Outlier List:
[ 15.  18. 297. 305.]

Nonoutlier range in Loudness (dB): (-19.97, 1.95)
Outlier List:
[-27. -24. -24. -22. -22. -22. -22. -22. -22. -21. -21. -21. -21. -21.
 -21. -21. -21. -20. -20. -20. -20. -20.]

Nonoutlier range in Liveness: (-31.23, 69.27)
Outlier List:
[70. 70. 70. 71. 71. 72. 72. 73. 73. 73. 73. 76. 76. 76. 76. 76. 77. 77.
 78. 78. 79. 80. 81. 81. 82. 83. 83. 84. 85. 85. 85. 85. 86. 87. 87. 89.
 90. 91. 91. 92. 92. 93. 93. 94. 95. 95. 96. 96. 96. 97. 97. 97. 97. 97.
 98. 99. 99.]

Nonoutlier range in Length (Duration): (-18.84, 544.08)
Outlier List:
[ 559.  564.  575.  576.  581.  590.  598.  598.  602.  620.  629.  639.
  653.  659.  663.  671.  703.  715.  809.  811.  859.  966. 1121. 1292.
 1367. 1412.]

Nonoutlier range in Speechiness: (-8.23, 18.21)
Outlier List:
[19. 19. 19. 19. 19. 20. 20. 20. 20. 22. 23. 23. 23. 24. 24. 24. 24. 25.
 25. 26. 26. 27. 28. 29. 30. 30. 30. 30. 31. 31. 31. 32. 32. 32. 

In [165]:
### 4. Missing Values ###

#Without using scikit-learn
# for column in songs.columns:
#     if column in ["Index","Title"]:
#         continue
#     elif column in ["Artist","Top Genre"]:
#         songs[column].fillna(songs[column].mode(),inplace=True)
#     else:
#         songs[column].fillna(songs[column].mean(),inplace=True)

# Replace categorical columns with the mode
cat_cols=list(songs.select_dtypes('object').columns)
imp_cat=SimpleImputer(missing_values=np.nan, strategy='most_frequent')
songs[cat_cols]=imp_cat.fit_transform(songs[cat_cols])

# Replace continuous columns with the mean
con_cols=list(songs.select_dtypes(['float64','int64']).columns)
imp_con=SimpleImputer(missing_values=np.nan, strategy='mean')
songs[con_cols]=imp_con.fit_transform(songs[con_cols])

songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987 entries, 0 to 1993
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Index                   1987 non-null   float64
 1   Title                   1987 non-null   object 
 2   Artist                  1987 non-null   object 
 3   Top Genre               1987 non-null   object 
 4   Year                    1987 non-null   float64
 5   Month                   1987 non-null   float64
 6   Beats Per Minute (BPM)  1987 non-null   float64
 7   Energy                  1987 non-null   float64
 8   Loudness (dB)           1987 non-null   float64
 9   Liveness                1987 non-null   float64
 10  Length (Duration)       1987 non-null   float64
 11  Acousticness            1987 non-null   float64
 12  Speechiness             1987 non-null   float64
 13  Popularity              1987 non-null   float64
dtypes: float64(11), object(3)
memory usage: 

In [166]:
### 5. Create an Age column and drop the other date columns ###

songs['Date']=pd.to_datetime(songs['Year'].apply(lambda x: str(round(x)))+'-'+songs['Month'].apply(lambda x: str(round(x)).zfill(2))+'-01',format='%Y-%m-%d')
songs['Age']=(dt.datetime.today()-songs['Date']).apply(lambda x: x.days/365)
songs.drop(columns=['Date','Year','Month'],inplace=True)
songs['Age']

0       17.306849
1       21.895890
2       20.643836
3       15.054795
4       19.476712
          ...    
1989    63.841096
1990    63.673973
1991    62.841096
1992    62.339726
1993    62.673973
Name: Age, Length: 1987, dtype: float64

In [167]:
### 6. Create dummy variables for the categorical features ###
#Drop Title?
#songs.drop('Title',inplace=True)

songs=pd.get_dummies(songs,drop_first=True)


### THIS IS THE END OF EXERCISE: DATA PROCESSING 